In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls

In [ ]:
# move into project directory
repo_name = "Image-Colorization"
%cd /content/drive/MyDrive/Personal-Projects/$repo_name
!ls

In [ ]:
# set up environment
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib numpy pandas pyyaml opencv-python

In [ ]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder


In [ ]:
# setup some imports
#custom imports
from transforms.transforms import ToTensor
from dataloading.datareader import DataReader
from dataloading.dataset import CustomDataset
from common.utils import get_exp_params, init_config, get_config

#py imports
import random
import numpy as np
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
# read experiment parameters
exp_params = get_exp_params()
print('Experiment parameters\n')
print(exp_params)

In [ ]:
# initialize directories and config data
init_config()
config = get_config()
print('Config parameters\n')
print(config)

In [ ]:
#initialize randomness seed
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
#preprocess data or load preprocessed data


#transform data

#convert to dataset

#load data